# Cohere Reranker

このノートブックでは、[Cohereのrerankエンドポイント](https://docs.cohere.com/docs/reranking)を使ってリトリーバーを作成する方法を紹介します。これは、[ContextualCompressionRetriever](contextual-compression.ipynb)でのアイデアをベースにしています。

In [1]:
# ドキュメントを表示するためのヘルパー関数

def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))

## ベクトルストアのベースとなるレトリーバーをセットアップする
まずはシンプルなベクトルストア・レトリーバーを初期化し、2023年の一般教書演説を（チャンクで）格納することから始めましょう。レトリーバーが高い数（20個）のドキュメントを検索するように設定することができます。

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS

documents = TextLoader('../../../state_of_the_union.txt').load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
texts = text_splitter.split_documents(documents)
retriever = FAISS.from_documents(texts, OpenAIEmbeddings()).as_retriever(search_kwargs={"k": 20})

query = "What did the president say about Ketanji Brown Jackson"
docs = retriever.get_relevant_documents(query)
pretty_print_docs(docs)

Document 1:

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.
----------------------------------------------------------------------------------------------------
Document 2:

As I said last year, especially to our younger transgender Americans, I will always have your back as your President, so you can be yourself and reach your God-given potential. 

While it often appears that we never agree, that isn’t true. I signed 80 bipartisan bills into law last year. From preventing government shutdowns to protecting Asian-Americans from still-too-common hate crimes to reforming military justice.
----------------------------------------------------------------------------------------------------
Document 3:

A for

## CohereRerankを使ったリランキングの実行
それでは、基本的なリトリーバーを`ContextualCompressionRetriever`でラップしましょう。ここに`CohereRerank`を追加し、Cohereのrerankエンドポイントを使って返された結果を再ランク付けします。

In [4]:
from langchain.llms import OpenAI
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank

llm = OpenAI(temperature=0)
compressor = CohereRerank()
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriever)

compressed_docs = compression_retriever.get_relevant_documents("What did the president say about Ketanji Jackson Brown")
pretty_print_docs(compressed_docs)

ValidationError: 1 validation error for CohereRerank
__root__
  Did not find cohere_api_key, please add an environment variable `COHERE_API_KEY` which contains it, or pass  `cohere_api_key` as a named parameter. (type=value_error)

もちろん、このリトリーバをQAパイプライン内で使用することができます。

In [5]:
from langchain.chains import RetrievalQA

In [6]:
chain = RetrievalQA.from_chain_type(llm=OpenAI(temperature=0), retriever=compression_retriever)

NameError: name 'compression_retriever' is not defined

In [34]:
chain({"query": query})

{'query': 'What did the president say about Ketanji Brown Jackson',
 'result': " The president said that Ketanji Brown Jackson is one of the nation's top legal minds and that she is a consensus builder who has received a broad range of support from the Fraternal Order of Police to former judges appointed by Democrats and Republicans."}